In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.window import Window
import numpy as np

spark = SparkSession.builder.getOrCreate()

23/02/01 21:41:37 WARN Utils: Your hostname, Davis-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.12 instead (on interface en0)
23/02/01 21:41:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/01 21:41:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Descriptive Analysis

In [2]:
# read parquet file
df = spark.read.parquet("../Uber_Lyft_Cab_prices/df_final.parquet/")

In [3]:
df.show(5)

+--------+--------+-------------+----------------+-----+----------------+--------------------+------------+------------+----------------+----+-----------+-----+-----+------+--------+------+--------+-----+
|distance|cab_type|  destination|          source|price|surge_multiplier|                  id|  product_id|        name|             day|hour|day_of_week|month| temp|clouds|pressure|  rain|humidity| wind|
+--------+--------+-------------+----------------+-----+----------------+--------------------+------------+------------+----------------+----+-----------+-----+-----+------+--------+------+--------+-----+
|    0.44|    Lyft|North Station|Haymarket Square|  5.0|             1.0|424553bb-7174-41e...|   lyft_line|      Shared|2018-12-16 07:30|   7|     Sunday|   12|38.46|  0.29| 1022.25|   0.0|    0.76| 7.68|
|    0.44|    Lyft|North Station|Haymarket Square| 11.0|             1.0|4bd23055-6827-41c...|lyft_premier|         Lux|2018-11-27 00:00|   0|    Tuesday|   11|43.82|  0.99| 1002.5

In [5]:
cab_type_distinct = df.select("cab_type").distinct().collect()
cab_type_distinct = [row.cab_type for row in cab_type_distinct]
cab_type_distinct

['Lyft', 'Uber']

In [6]:
cat_cab_type = df.groupBy("cab_type").count().show()

+--------+------+
|cab_type| count|
+--------+------+
|    Lyft|561118|
|    Uber|603878|
+--------+------+



In [7]:
destinations = df.select("destination").distinct().collect()
destinations = [row.destination for row in destinations]
len(destinations)

12

In [8]:
each_destination = df.groupBy("destination").count().orderBy("count", ascending=False).show()

+--------------------+-----+
|         destination|count|
+--------------------+-----+
|  Financial District|98804|
|   Boston University|97858|
|       South Station|97392|
|            Back Bay|97354|
|           North End|97240|
|         Beacon Hill|97085|
|              Fenway|97035|
|            West End|96981|
|Northeastern Univ...|96917|
|    Haymarket Square|96688|
|       North Station|95924|
|    Theatre District|95718|
+--------------------+-----+



In [9]:
each_source = df.groupBy("source").count().orderBy("count", ascending=False).show()

+--------------------+-----+
|              source|count|
+--------------------+-----+
|  Financial District|99448|
|Northeastern Univ...|97334|
|         Beacon Hill|97318|
|    Haymarket Square|97171|
|            Back Bay|97165|
|           North End|97114|
|   Boston University|97005|
|       South Station|96951|
|            West End|96889|
|    Theatre District|96495|
|              Fenway|96291|
|       North Station|95815|
+--------------------+-----+



In [10]:
source_destination = df.groupBy("source", "destination").count().orderBy("count", ascending=False).show(50)

+--------------------+--------------------+-----+
|              source|         destination|count|
+--------------------+--------------------+-----+
|           North End|         Beacon Hill|17679|
|       South Station|  Financial District|17474|
|  Financial District|       South Station|17331|
|            Back Bay|           North End|17307|
|         Beacon Hill|           North End|17207|
|    Haymarket Square|  Financial District|17147|
|            West End|              Fenway|17106|
|              Fenway|            West End|17081|
|           North End|            Back Bay|17029|
|   Boston University|            West End|16992|
|  Financial District|              Fenway|16822|
|       South Station|    Theatre District|16765|
|    Theatre District|   Boston University|16749|
|  Financial District|   Boston University|16715|
|  Financial District|    Haymarket Square|16651|
|    Theatre District|    Haymarket Square|16642|
|       North Station|              Fenway|16587|
